# **MIP1_Run1**

In [1]:
!pip install openai
!pip install python-dotenv
!pip3 install pyomo
!apt install glpk-utils
!pip install glpk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.5/325.5 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 35.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 5.4 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libamd2 libcolamd2 libglpk40 libsuitesparseconfig5
Suggested packages:
  libiodbc2-dev
The following NEW packages will be installed:
  glpk-utils libamd2 libcolamd2 libglpk40 libsuitesparseconfig5
0 upgraded, 5 newly installed, 0 to remove and 45 not upgraded.
Need to get 625 kB of archives.
After this operation, 2,158 kB of additional disk space will be used.
Ge

In [2]:

import openai
import os
from IPython.display import Markdown
import pyomo


### **Accessing the GPT4 API**

In [ ]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv('api_file.env'))
my_api_key = os.environ['api_key_env']



### **Generate Mathematical Model**

In [ ]:
problem = """ You are a city planner, looking to open facilities at some locations. We have a set of customers and a set of possible locations for opening facilities. Each potential location for establishing a facility incurs a fixed annual activation cost, which must be paid if the facility is used, regardless of the service volume it handles. Furthermore, this service volume at each facility is capped at a maximum annual limit. Additionally, there are transportation costs associated with servicing each customer from each facility.
The goal is to minimize the overall costs, which include both the fixed activation costs for any opened facilities and the transportation costs for servicing customers. This must be done while making sure that each customer’s demand is met, each facility does not exceed its maximum service volume, and customers can of course only be serviced by a facility that is opened.
Please formulate this as a mathematical optimization model."""

In [ ]:
client = openai.OpenAI(api_key=os.environ['api_key_env'])

response1 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please formulate a mathematical optimization model for this problem. Include parameters, decision variables, the objective function and the constraints in your answer."},
        {
            "role": "user",
            "content": problem
    }
    ],
    model="gpt-4",
    seed = 1
)


In [ ]:
print(response1.choices[0].message.content)

Parameters:

Let's denote the given parameters as follows:

- i: index representing potential facility locations (i ∈ I).
- j: index representing customers (j ∈ J).
- F_i: fixed annual activation cost for facility (i).
- C_ij: the transportation cost from facility (i) to customer (j).
- D_j: the demand of customer (j).
- V_i: the maximum annual service volume of facility (i).

Decision Variables:

- x_i: binary variable that equals 1 if facility (i) is opened, and 0 otherwise.
- y_ij: continuous variable representing the service volume from facility (i) to customer (j).

Objective Function:

The objective is to minimize the overall annual costs, including both the fixed activation costs for opened facilities and transportation costs.

Min ∑_i∈I [ F_i * x_i + ∑_j∈J (C_ij * y_ij) ]

Constraints:

- Facility Service Constraints: Each opened facility should not surpass its maximum annual service volume: 
  ∑_j∈J y_ij ≤ V_i * x_i for all i

- Demand Satisfaction Constraints: The service ren

### **Generate Pyomo Code**

In [ ]:

response2 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please write a python pyomo code for this optimization problem. Use sample data where needed. Indicate where you use sample data."},
        {
            "role": "user",
            "content": response1.choices[0].message.content
        }
    ],
    model="gpt-4",
    seed = 1
)

In [ ]:
Markdown(response2.choices[0].message.content)

Here is a Pyomo code implementing your optimization problem. We'll presume you need to solve with the GLPK solver:

```python
from pyomo.environ import *

# Sample data
I = range(4)  # index for potential facilities
J = range(5)  # index for customers
F = {i: 10*(i+1) for i in I}  # fixed annual activation cost for each facility
V = {i: 100*(i+1) for i in I}  # max annual service volume for each facility
C = {(i,j): 2*(i+1)*(j+1) for i in I for j in J}  # transportation cost from facility i to customer j
D = {j: 50*(j+1) for j in J}  # demand of each customer

# Create a model
model = ConcreteModel()

# Variables
model.x = Var(I, within=Binary)
model.y = Var(I, J, within=NonNegativeReals)

# Objective
model.obj = Objective(expr = sum(F[i]*model.x[i] for i in I) + \
                      sum(C[i,j]*model.y[i,j] for i in I for j in J), 
                      sense=minimize)

# Constraints
model.facility_service = ConstraintList()
for i in I:
  model.facility_service.add(expr = sum(model.y[i,j] for j in J) <= V[i]*model.x[i])
  
model.demand_satisfaction = ConstraintList()
for j in J:
  model.demand_satisfaction.add(expr = sum(model.y[i,j] for i in I) == D[j])

model.facility_activation = ConstraintList()
for i in I:
  for j in J:
    model.facility_activation.add(model.y[i,j] <= V[i]*model.x[i])

# Solve
solver = SolverFactory('glpk')
solver.solve(model)

# Display the solution
for i in I:
  print('x[', i, '] =', model.x[i].value)
for i in I:
  for j in J:
    print('y[', i, ',', j, '] =', model.y[i,j].value)
```
Please replace the "sample data" with your actual data or adjust the sample data to meet your needs. You also need to have GLPK solver installed in your system and make it accessible for Pyomo.

In this code, ConcreteModel creates an instance of a problem. Var, Objective, and ConstraintList are employed to create decision variables, the objective function, and the constraints respectively. The SolverFactory('glpk') line specifies that the GLPK solver should be used. Finally, the "solver.solve(model)" line actually solves the optimization problem.

### **Run the code Generated by GPT4**

In [ ]:
from pyomo.environ import *

# Sample data
I = range(4)  # index for potential facilities
J = range(5)  # index for customers
F = {i: 10*(i+1) for i in I}  # fixed annual activation cost for each facility
V = {i: 100*(i+1) for i in I}  # max annual service volume for each facility
C = {(i,j): 2*(i+1)*(j+1) for i in I for j in J}  # transportation cost from facility i to customer j
D = {j: 50*(j+1) for j in J}  # demand of each customer

# Create a model
model = ConcreteModel()

# Variables
model.x = Var(I, within=Binary)
model.y = Var(I, J, within=NonNegativeReals)

# Objective
model.obj = Objective(expr = sum(F[i]*model.x[i] for i in I) + \
                      sum(C[i,j]*model.y[i,j] for i in I for j in J),
                      sense=minimize)

# Constraints
model.facility_service = ConstraintList()
for i in I:
  model.facility_service.add(expr = sum(model.y[i,j] for j in J) <= V[i]*model.x[i])

model.demand_satisfaction = ConstraintList()
for j in J:
  model.demand_satisfaction.add(expr = sum(model.y[i,j] for i in I) == D[j])

model.facility_activation = ConstraintList()
for i in I:
  for j in J:
    model.facility_activation.add(model.y[i,j] <= V[i]*model.x[i])

# Solve
solver = SolverFactory('glpk')
solver.solve(model)

# Display the solution
for i in I:
  print('x[', i, '] =', model.x[i].value)
for i in I:
  for j in J:
    print('y[', i, ',', j, '] =', model.y[i,j].value)

x[ 0 ] = 1.0
x[ 1 ] = 1.0
x[ 2 ] = 1.0
x[ 3 ] = 1.0
y[ 0 , 0 ] = 0.0
y[ 0 , 1 ] = 0.0
y[ 0 , 2 ] = 0.0
y[ 0 , 3 ] = 0.0
y[ 0 , 4 ] = 100.0
y[ 1 , 0 ] = 0.0
y[ 1 , 1 ] = 0.0
y[ 1 , 2 ] = 0.0
y[ 1 , 3 ] = 50.0
y[ 1 , 4 ] = 150.0
y[ 2 , 0 ] = 0.0
y[ 2 , 1 ] = 0.0
y[ 2 , 2 ] = 150.0
y[ 2 , 3 ] = 150.0
y[ 2 , 4 ] = 0.0
y[ 3 , 0 ] = 50.0
y[ 3 , 1 ] = 100.0
y[ 3 , 2 ] = 0.0
y[ 3 , 3 ] = 0.0
y[ 3 , 4 ] = 0.0


In [4]:
from pyomo.environ import *

# data inputted by human
J = range(1,6)  # index for potential facilities
I = range(1, 4)  # index for customers
F = {1: 1000, 2: 1000, 3: 1000} # fixed annual activation cost for each facility
V = {1: 500, 2: 500, 3: 500}  # max annual service volume for each facility
C = {
    (1, 1): 4,  (2, 1): 6,  (3, 1): 9,
    (1, 2): 5,  (2, 2): 4,  (3, 2): 7,
    (1, 3): 6,  (2, 3): 3,  (3, 3): 4,
    (1,4): 8,  (2,4): 5,  (3,4): 3,
    (1,5): 10, (2,5): 8,  (3,5): 4,
}
    # transportation cost from facility i to customer j
D = {1: 80, 2: 270, 3: 250, 4: 160, 5: 180} # demand of each customer
#end
# Create a model
model = ConcreteModel()

# Variables
model.x = Var(I, within=Binary)
model.y = Var(I, J, within=NonNegativeReals)

# Objective
model.obj = Objective(expr = sum(F[i]*model.x[i] for i in I) + \
                      sum(C[i,j]*model.y[i,j] for i in I for j in J),
                      sense=minimize)

# Constraints
model.facility_service = ConstraintList()
for i in I:
  model.facility_service.add(expr = sum(model.y[i,j] for j in J) <= V[i]*model.x[i])

model.demand_satisfaction = ConstraintList()
for j in J:
  model.demand_satisfaction.add(expr = sum(model.y[i,j] for i in I) == D[j])

model.facility_activation = ConstraintList()
for i in I:
  for j in J:
    model.facility_activation.add(model.y[i,j] <= V[i]*model.x[i])

# Solve
solver = SolverFactory('glpk')
solver.solve(model)

# Display the solution
print(model.obj())
for i in I:
  print('x[', i, '] =', model.x[i].value)
for i in I:
  for j in J:
    print('y[', i, ',', j, '] =', model.y[i,j].value)

5610.0
x[ 1 ] = 0.0
x[ 2 ] = 1.0
x[ 3 ] = 1.0
y[ 1 , 1 ] = 0.0
y[ 1 , 2 ] = 0.0
y[ 1 , 3 ] = 0.0
y[ 1 , 4 ] = 0.0
y[ 1 , 5 ] = 0.0
y[ 2 , 1 ] = 80.0
y[ 2 , 2 ] = 270.0
y[ 2 , 3 ] = 150.0
y[ 2 , 4 ] = 0.0
y[ 2 , 5 ] = 0.0
y[ 3 , 1 ] = 0.0
y[ 3 , 2 ] = 0.0
y[ 3 , 3 ] = 100.0
y[ 3 , 4 ] = 160.0
y[ 3 , 5 ] = 180.0


### **Edit and Run the code for the mathematical model produced by GPT4 (Circumstantial)**